In [64]:
#IMPORTA LIBRERIAS
#====================================================#

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from googletrans import Translator, constants
import pickle

In [12]:
#DEFINIENDO THRESHOLD 
#==================================================================================#

class_names = ['TOXIC','SEVERE TOXIC','OBSCENE','THREAT','INSULT','IDENTITY HATE']
thresholds = [0.4,0.15,0.15,0.01,0.1,0.01]

In [3]:
#CARGAR ARCHIVO TOKENIZER
#======================================================================================#

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [9]:
#CARGANDO MEJOR MODELO
#======================================================================================#

model = load_model('checkpoints/cp-toxic_sen10.hdf5')

In [88]:
#LIMPIAR TEXTO DE ENTRADA
#========================================================================================#

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [101]:
#FUNCION PREDICTORA
#=========================================================================================#

translator = Translator()
def predict(input_text):
    input_text = preprocess_text(input_text)
    
    input_text = translator.translate(preprocess_text(input_text)).text
    
    text_tokenizer = tokenizer.texts_to_sequences([input_text])
    text_padd = pad_sequences(text_tokenizer, padding='post', maxlen=maxlen)
    
    pred = model.predict(text_padd)
    
    class_preds = []
    class_names_preds = []
    for i,p in enumerate(pred[0]):
        thr = thresholds[i]
        if p > thr:
            class_preds.append(1)
            class_names_preds.append(class_names[i])
        else:
            class_preds.append(0)
    return class_preds, class_names_preds

In [112]:
#MOSTRAR PREDICCIONES DADO UN COMENTARIO DE ENTRADA
#==============================================================================#
def printPrediction(input_text):
    preds = predict(input_text)
    print("\ncomment: ", input_text)
    print("class: ",preds[0])
    if len(preds[1]) == 0:
        print("names: ","ninguno")
        return
    print("names: ", ', '.join(n for n in preds[1]))

In [117]:
#COMENTARIOS DE ENTRADA PARA VALIDAR EL MODELO
#==============================================================================================#

texts = ["I'm going to hit you if you do not cooperate with me",
         "maravilloso lugar",  
         "me senti como en cas",
         "i also sock puppet of this account suprise sincerely the man that will track you down from the internet and kill you",
         "hay personas que lo hacen mejor que tu lo haces horrible es una completa mierda deberias retirarte"]

for t in texts:
    printPrediction(t)


comment:  I'm going to hit you if you do not cooperate with me
class:  [1, 0, 0, 1, 0, 1]
names:  TOXIC, THREAT, IDENTITY HATE

comment:  maravilloso lugar
class:  [0, 0, 0, 0, 0, 0]
names:  ninguno

comment:  me senti como en cas
class:  [0, 0, 0, 0, 0, 0]
names:  ninguno

comment:  i also sock puppet of this account suprise sincerely the man that will track you down from the internet and kill you
class:  [1, 0, 0, 1, 1, 1]
names:  TOXIC, THREAT, INSULT, IDENTITY HATE

comment:  hay personas que lo hacen mejor que tu lo haces horrible es una completa mierda deberias retirarte
class:  [1, 0, 1, 0, 1, 1]
names:  TOXIC, OBSCENE, INSULT, IDENTITY HATE


### CONCLUSIONES

como se observa el modelo funciona bastante bien y logra identificar los comentarios mal intencionados. los umbrales de decisión por cada clase parecen ser los correctos.

como mejora se pueden aumentar los comentarios para las etiquetas 1, y asi aumentar la presición en identificar este tipo de comentarios